## Agents
### Your Personal Health Expert 

- Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be.
- The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to finish.

In [ ]:
from dotenv import load_dotenv
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

load_dotenv()

In [ ]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool



llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
llm.invoke('hi')

### Define Tools
#### 1. Tavily

In [3]:
from langchain_community.tools import TavilySearchResults

@tool
def search(query: str) -> str:
    """Search Tavily for current and latest news and updates.
        For example, stock market news, weather updates, news etc.
    Args:
        query: The search query.
    """
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True
    )
    response = search.invoke(query)
    return response

In [ ]:
# tavily_search("stock market news")
search.invoke("stock market news")

#### 2. Health Supplements Retriever

In [5]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

db_name = r"D:\NLP\LLM\Langchain and Ollama\09. Vector Stores and Retrievals\health_supplements"
vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

In [6]:
@tool
def health_supplements(query: str) -> str:
    """Search for information about Health Supplements.
    For any questions about Health and Gym Supplements, you must use this tool!,

    Args:
        query: The search query.
    """
    response = retriever.invoke(query)
    return response

In [ ]:
question = "What is the best supplement for muscle gain?"
health_supplements.invoke(question)

### Agent Creation

 - The agent is responsible for taking in input and deciding what actions to take
 - the Agent does not execute those actions - that is done by the AgentExecutor
 - `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

In [ ]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [11]:
tools = [search, health_supplements]
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
question = "What is the best supplement for muscle gain?"
# question = "What's the weather in SF?"

response = agent_executor.invoke({'input': question})
print(response['input'])
print("\n\nOutput:")
print(response['output'])

In [ ]:
question = "What's the weather in SF?"

response = agent_executor.invoke({'input': question})
print(response['input'])
print("\n\nOutput:")
print(response['output'])

In [ ]:
question = "Write Report on the stock market news"

response = agent_executor.invoke({'input': question})
print(response['input'])
print("\n\nOutput:")
print(response['output'])